<h2>Ejemplo de Text-Scraping para Dani!!</h2>
<p>Hola Dani, esta ma&ntilde;ana me ha dado por crearte un ejemplo r&aacute;pido y sucio (<span data-dobid="hdw">quick and dirty)</span>&nbsp;para ense&ntilde;arte como hacer un Text scraping. Para ello vamos a 'scrapear' la p&aacute;gina:</p>
<p>&nbsp;<a href="https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/salud/faq.htm">https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/salud/faq.htm</a></p>
<p>&nbsp;</p>
<p>Que contiene las y preguntas y respuestas m&aacute;s habituales en un Servicio de Vacunaci&oacute;n Internacional&nbsp;</p>
<p>Con ellas vamos a crear una tabla</p>
<p> Son pocas preguntas (50) y seguramente con un copy paste iríamos más rapido, pero se trata simplemente un ejemplo ilustrativo</p>

<p> Importamos las librerias que necesitamos para el scraping</p>
<ul>
    <li> Requests: para hacer los http requests</li>
    <li> BeutifulSoup: para hacer el scraping</li>
    <li>pickle: para guardar la información</li>
</ul>

<p>definimos una función dónde le damos una URL y extraemos la información de la clase "informacion" y de los elementos h2, h3...</p> 
<p> Esto lo hacemos tras analizar la página y ver dónde se encuentra la información que necesitamos!</p>

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
import pickle

url = 'https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/salud/faq.htm'

page = requests.get(url,verify=False).text
soup = BeautifulSoup(page,"html.parser")

text = [p.text for p in soup.find (class_="informacion").find_all(["h2","h3", "p", "ol","ul", "li"])]
text

In [ ]:
#Removing last element of the list! we don't need the text: Si desea localizar información... !!!
text.pop()
text

<h3>Guardamos y exportamos la informaci&oacute;n en bruto por si queremos analizar otro dia más cosas</h3>
<p>Para ello utilizamos pickle</p>

In [ ]:
#we save the raw data using pickle
with open("preguntas-gob.txt", "wb") as file:
    data=pickle.dump(text,file)

In [ ]:
#read the pickle
with open ("preguntas-gob.txt", "rb") as file:
    data = pickle.load(file)

Combinamos la información para poder analizar todo el texto como si fuera uno:

In [ ]:
combined_text = ' '.join(data)

combined_text

<p>En el texto nos encontramos cosas raras como:</p>
<p>'<em> siguiente enlace del Ministerio de Asuntos Exteriores y Cooperaci&oacute;n. \nSi Viajas al extranjero (Ministerio de Asuntos Exteriores y Cooperaci&oacute;n)<strong>\n\t\t\t\t\t\t\t\t\t\t \t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\n</strong> &iquest;Qu&eacute; debemos tener en cuenta de la Asistencia sanitaria en el extranjero? En los pa&iacute;ses de la Comunidad Europea, la Tarjeta Sanitaria Europea es el documento emitido por el Instituto Nacional de la Seguridad Social (INSS)</em> '</p>
<p>Que queremos eliminar!</p>

<p>Para ello utilzamos 'expresiones regulares (regex)' </p>

In [ ]:
import re
regex = re.compile(r'[\n\t]')
plaintext= regex.sub(" ", combined_text)


<p>Una vez tenemos el texto limpio lo queremos separar por pregunta - respuesta </p>
<p> para ello creamos una lista y añadimos un nuevo elemento cada vez que nos encontremos "¿"</p> 

In [ ]:
questions_answers= ["¿"+ e for e in plaintext.split("¿") if e]
questions_answers

<p>Aqui me he complicado un poco, pero simplemente:<p>
    <ol>
        <li>Separamos preguntas de respuestas</li>
        <li>Las metemos en un diccionario</li>
    </ol>

In [ ]:
question_dicc = {}
for question in questions_answers:
    pr = question.split('?')
    question_dicc [questions_answers.index(question)] = [pr[0]+'?', pr[1].strip()]


question_dicc


<p> Para visualizarlo creamos un Dataframe de Pandas</p>

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 3000)
data_df = pd.DataFrame.from_dict(question_dicc).transpose()
data_df.columns = ['questions', 'answers']
data_df

<p> Exportamos la tabla en un CSV!! </p>

In [ ]:
data_df.to_csv(r'preguntas-respuestas-gob.csv',index = False )

<h2>Segunda Parte!!</h2>

En esta segunda parte vamos a ver un ejemplo de PDF-Scraping!

vamos a scrapear algunos pdfs de 
https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/salud/infVacunas.htm

para ello vamos a instalar pdfminer! 

Todo el notebook se ha realizado de forma rápida para mostrar un ejemplo! Si hay mejoras, porfavor! no dudeis en mejorarlo :)


In [ ]:
pip install pdfminer.six

Definimos una funcion para guardar los pdfs:

In [ ]:
import requests

 
def save_pdf_from_url(url,pathname):    
    response = requests.get(url,verify=False, stream=True)
    with open(pathname, 'wb') as f:
        f.write(response.content)

Y definimos las urls y guardamos los pdfs: 

In [ ]:
url1 = "https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/docs/COLERA.pdf"
pathname1="COLERA.pdf"

url2 = "https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/docs/ENCEFALITIS_CENTRO_EUROPEA_.pdf"
pathname2="ENCEFALITIS_CENTRO_EUROPEA.pdf"

url3 = "https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/docs/FIEBRE_AMARILLA.pdf"
pathname3="FIEBRE_AMARILLA.pdf"

url4 = "https://www.mscbs.gob.es/profesionales/saludPublica/sanidadExterior/docs/FIEBRE_TIFOIDEA_INYECTABLE.pdf"
pathname4="FIEBRE_TIFOIDEA_INYECTABLE.pdf"


urls = [[url1,pathname1],[url2,pathname2],[url3,pathname3],[url4,pathname4] ]

for url in urls:    
    save_pdf_from_url(url[0],url[1])

Definimos una función para extraer el texto de los pdfs:
(reutilizo el ejemplo de https://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library)

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO


def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

Extraemos el texto de los PDFS

In [ ]:
textlist = []

for url in urls:
    print(url[1])
    textlist.append((url[1].replace(".pdf",""), convert_pdf_to_txt(url[1])))
    

In [ ]:
textlist

Limpiamos un poco el texto, aquí se han hecho varias iteraciones

In [ ]:
import re

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text.strip()

to_replace_1 = r"Información para el Viajero\nInformación para el Viajero\nEncefalitis Centro-Europea\n\n"
to_replace_2= r"\n\n     Consulte con un\nCentro de\nVacunación\nInternacional\n\nwww.msssi.gob.es/cvi\n\n901 400 100\n\n\x0c"
to_replace_3=r"\n"
to_replace_4=r"\x0c"
to_replace_5=r"Consulte con un\n\nCentro de\nVacunación\nInternacional\n\nwww.msssi.gob.es/cvi\n\n901 400 100"
to_replace_6=r"(«.*?»)"
to_replace_7=r"(«.*?«)"
to_replace_8=r"VacunaCólera     Consulte con unCentro deVacunaciónInternacionalwww.msssi.gob.es/cvi901 400 100"
to_replace_9=r"Información para el ViajeroInformación para el ViajeroFiebre Amarilla"

#to_replace=[to_replace_1,to_replace_2,to_replace_3,to_replace_4]
to_replace=[to_replace_1,to_replace_2,to_replace_3,to_replace_4,to_replace_5,to_replace_6,to_replace_7,
           to_replace_8,to_replace_9]


cleaned_list =[]

for t in textlist:
    print(t[1])
    cleaned_list.append([t[0],clean_text(to_replace,t[1])])
    

cleaned_list

Esta funcion definida, hace lo que se ha hecho en la parte 1:
    separamos preguntas y respuestas y creamos un diccionario

In [ ]:
# function to do what i've done

def prepare_dicc(text):
    counter=0
    questions_answers_dict = dict()
    answers=[]
    answers= ["¿"+ e for e in text.split("¿") if e]
    
    for question in answers:
        pr = question.split('?')
        #I'm cleaning the answers to be sure we don't get some sound/mierda
        ans= clean_text(to_replace,pr[1].strip())
        questions_answers_dict [counter] = [pr[0]+'?', ans]
        counter=counter+1
    return questions_answers_dict



Analizamos que preguntas hay en el primer elemento de la lista para crear un diccionario

In [ ]:
nombre_1 = cleaned_list[0][0]
preguntas_1=cleaned_list[0][1]
#identifico que voy a ver
nombre_1

creamos un diccionario:

In [ ]:
colera_dict = prepare_dicc(preguntas_1)
colera_dict

y por último, utilizamos Pandas para ver la tabla:

In [ ]:
pd.set_option('max_colwidth', 3000)
data_df = pd.DataFrame.from_dict(colera_dict).transpose()
data_df.columns = ['questions', 'answers']
data_df

In [ ]:
nombre_2 = cleaned_list[1][0]
preguntas_2=cleaned_list[1][1]

ENCEFALITIS_CENTRO_EUROPEA_dict = prepare_dicc(preguntas_2)

pd.set_option('max_colwidth', 3000)
data_df = pd.DataFrame.from_dict(ENCEFALITIS_CENTRO_EUROPEA_dict).transpose()
data_df.columns = ['questions', 'answers']
data_df

In [ ]:
nombre_3 = cleaned_list[2][0]
preguntas_3=cleaned_list[2][1]

FIEBRE_AMARILLA_dict = prepare_dicc(preguntas_3)

pd.set_option('max_colwidth', 3000)
data_df = pd.DataFrame.from_dict(FIEBRE_AMARILLA_dict).transpose()
data_df.columns = ['questions', 'answers']
data_df



In [ ]:
nombre_4 = cleaned_list[3][0]
preguntas_4=cleaned_list[3][1]

FIEBRE_TIFOIDEA_dict = prepare_dicc(preguntas_4)

pd.set_option('max_colwidth', 3000)
data_df = pd.DataFrame.from_dict(FIEBRE_TIFOIDEA_dict).transpose()
data_df.columns = ['questions', 'answers']
data_df